here I'm using a custom [scallop](https://github.com/Kingsford-Group/scallop) + [gtfmerge](https://github.com/Kingsford-Group/rnaseqtools#gtfmerge) pipeline to create merged gtf annotation of hg19 and tinat transcripts introduced in this 

Alex
> I was curious myself so I decided to align our RNA-seq data to their chimeric transcripts... it looks promising, but let me know if there's anything in my analysis I need to change. Still working on seeing if we can identify alternative TSS without CAGE-seq. 
I'm guessing we can look for novel transcripts using [StringTie](https://ccb.jhu.edu/software/stringtie/) or cufflinks?

Ray
> I'm asking because in [this paper](https://www.nature.com/articles/ng.3889) they treated lung cancer cells with decitabine and performed Cage-seq to map and identify cryptic transcription start sites. They found cryptic TSSs that generated spliced variants, half of which were in-frame isoforms (and some of which resulted in chimeric transcripts, others original or truncated). This got me thinking about our project and whether we could see these chimeric/fusion transcripts (that are skewed toward the 5'end due Cage-seq and being TSS-induced) in HL60s.  


Luke
> Hi Ray, Alex and Abe
This is an interesting paper which claims genetic alterations to DNA methylation core genes drive dysregulated hematopoietic development due to CpG changes that regulate transcription factor binding (figure 4 and 5). How many TFs are methylation sensitive has been controversial in my understanding but is an interesting idea with lots of data within mylomonocytic cells. https://www.nature.com/articles/s41588-020-0595-4
I am not suggesting a specific plan but this is an interesting idea.

___
last update    

    Thu Oct 21 01:06:19 UTC 2021



In [2]:
cat ../../scallop-genome/gtf-list.txt

gencode.v34lift37.basic.annotation.gtf
tinat/DAC_TINATs.gtf


In [3]:
# cat scallop/tinat/tinat-gtf-list.txt
!wc -l ../../scallop-genome/tinat/DAC+SB_TINATs.gtf
!wc -l ../../scallop-genome/tinat/DAC_TINATs.gtf
!wc -l ../../scallop-genome/tinat/SB939_TINATs.gtf

12172 ../../scallop-genome/tinat/DAC+SB_TINATs.gtf
1403 ../../scallop-genome/tinat/DAC_TINATs.gtf
2059 ../../scallop-genome/tinat/SB939_TINATs.gtf


In [4]:
cat ../../scallop-genome/tinat.sh

# https://github.com/Kingsford-Group/scallop
mkdir -p scallop/tinat/compare

# export PATH='/sadra/goodarzilab/abe/Workflows/rnaseqtools-1.0.3/bin/':$PATH

conda activate alignment
cd scallop-genome/

### 0. download genomes
# # mkdir -p hg19
# # wget https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/genes/hg19.knownGene.gtf.gz
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/GRCh37_mapping/gencode.v34lift37.basic.annotation.gtf.gz
gunzip gencode.v34lift37.basic.annotation.gtf.gz
# # mv hg19.knownGene.gtf.gz hg19/hg19.knownGene.gtf.gz
# # gunzip hg19/hg19.knownGene.gtf.gz

### 1. merge
gtfmerge union gtf-list.txt DAC_GRCh37_merged.gtf -t 2

# ### 2. compare 
# gffcompare -o hg19_tinat/gffall -r /rumi/shams/genomes/hg19/hg19_genes.gtf tinat/hg19.tinat.gtf 

# ### 3. subset
# gtfcuff puniq \
#     tinat/gffall.hg19.tinat.gtf.tmap \
#     tinat/hg19.tinat.gtf \
#     /rumi/shams/genomes/hg19/hg19_genes.gtf \
#     tinat/unique.gtf

### 4. gtf2fasta
# gffread

In [13]:
%%bash 
mkdir -p tinat/quants

for f in fastq/*; do
	f=`basename $f`;
	o=${f/.fastq.gz/};
	echo $o
	echo `date`
    salmon quant \
        -i ../../scallop-genome/salmon.index -l A \
        -r fastq/$f -o tinat/quants/$o -p 20;
	echo `date`
done

hl60_120h_t_1
Mon Sep 4 00:20:28 PDT 2023
Mon Sep 4 00:21:18 PDT 2023
hl60_120h_t_2
Mon Sep 4 00:21:18 PDT 2023
Mon Sep 4 00:22:10 PDT 2023
hl60_120h_u_1
Mon Sep 4 00:22:10 PDT 2023
Mon Sep 4 00:22:59 PDT 2023
hl60_120h_u_2
Mon Sep 4 00:22:59 PDT 2023
Mon Sep 4 00:23:53 PDT 2023
hl60_6h_t_1
Mon Sep 4 00:23:53 PDT 2023
Mon Sep 4 00:24:32 PDT 2023
hl60_6h_t_2
Mon Sep 4 00:24:32 PDT 2023
Mon Sep 4 00:25:41 PDT 2023
hl60_6h_u_1
Mon Sep 4 00:25:41 PDT 2023
Mon Sep 4 00:26:19 PDT 2023
hl60_6h_u_2
Mon Sep 4 00:26:19 PDT 2023
Mon Sep 4 00:27:08 PDT 2023
hl60_72h_t_1
Mon Sep 4 00:27:08 PDT 2023
Mon Sep 4 00:27:59 PDT 2023
hl60_72h_t_2
Mon Sep 4 00:27:59 PDT 2023
Mon Sep 4 00:28:49 PDT 2023
hl60_72h_u_1
Mon Sep 4 00:28:49 PDT 2023
Mon Sep 4 00:29:49 PDT 2023
hl60_72h_u_2
Mon Sep 4 00:29:49 PDT 2023
Mon Sep 4 00:31:10 PDT 2023
kg1_t_1
Mon Sep 4 00:31:10 PDT 2023
Mon Sep 4 00:32:10 PDT 2023
kg1_t_2
Mon Sep 4 00:32:10 PDT 2023
Mon Sep 4 00:32:26 PDT 2023
kg1_t_3
Mon Sep 4 00:32:26 PDT 2023
Mon Sep 

Version Info: Could not resolve upgrade information in the alotted time.
Check for upgrades manually at https://combine-lab.github.io/salmon
### salmon (mapping-based) v1.2.1
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { ../../scallop-genome/salmon.index }
### [ libType ] => { A }
### [ unmatedReads ] => { fastq/hl60_120h_t_1.fastq.gz }
### [ output ] => { tinat/quants/hl60_120h_t_1 }
### [ threads ] => { 20 }
Logs will be written to tinat/quants/hl60_120h_t_1/logs
[2023-09-04 00:20:29.348] [jointLog] [info] setting maxHashResizeThreads to 20
[2023-09-04 00:20:29.348] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2023-09-04 00:20:29.348] [jointLog] [info] Usage of --validateMappings implies use of minScoreFraction. Since not explicitly specified, it is being set to 0.65
[2023-09-04 00:20:29.348] [jointLog] [info] Usage of --validateMappings implies a default consensus slack of 0.2. Setting consensus

In [18]:
rm -r tinat/quants/ociaml2*
rm -r tinat/quants/thp1*

In [15]:
!date

Mon Sep  4 01:42:27 PDT 2023
